In [1]:
import pandas as pd
import numpy as np
from itertools import product
from pandas import Series
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, Normalizer, OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn_pandas import DataFrameMapper

In [2]:
DATAFILE = 'data/LoanStats_20{year}Q{quarter}.csv'
dframes = list()
for year, quarter in product([16, 17, 18], [1, 2, 3, 4]):
    temp_df = pd.read_csv(DATAFILE.format(year=year, quarter=quarter), 
                            sep=",", skiprows=1,)# nrows=10000)
    dframes.append(temp_df)
loan_data = pd.concat(dframes, axis=0, ignore_index=True)

/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,112) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-pack

In [3]:
columns = [
"loan_amnt",
"int_rate",
"grade",
"sub_grade",
"emp_length",
"home_ownership",
"annual_inc",
"verification_status",
"loan_status",
"dti",
"open_acc",
"pub_rec",
"revol_bal",
"revol_util",
"total_acc",
]
loan_data = loan_data[columns]
loan_data['revol_util'] = loan_data['revol_util'].str.replace("%","").astype(float) / 100
loan_data['int_rate'] = loan_data['int_rate'].str.replace("%","").astype(float) / 100
loan_data = loan_data[np.isfinite(loan_data['loan_amnt'])]
loan_data.groupby(['emp_length']).count()
loan_data = loan_data[loan_data.loan_status.isin(['Fully Paid','Charged Off'])]

In [4]:
loan_data = loan_data.dropna()

In [5]:
def emp_length(x: str):
    emps = ['10+ years', '< 1 year', '1 year', '2 years', '3 years', '4 years',
           '5 years','6 years', '7 years', '8 years', '9 years', 'placeholder','10+ years']
    try:
        i = emps.index(x)
        return i
    except:
        return -1

loan_data.emp_length = loan_data.emp_length.apply(emp_length)


In [6]:
normalizers = ['emp_length','int_rate','loan_amnt', 'annual_inc', 'dti', 'open_acc', 'pub_rec','revol_bal','revol_util', 'total_acc']
ordinals = ['grade', 'sub_grade', 'loan_status']
one_hot = ['home_ownership', 'verification_status']
for ordinal in ordinals + one_hot:
    loan_data[ordinal] = LabelEncoder().fit_transform(loan_data[ordinal].values.reshape(-1,1))
for norm in normalizers:
    loan_data[norm] = StandardScaler().fit_transform(loan_data[norm].values.reshape(-1,1))

/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
from sklearn import svm
clf = svm.SVC()
Y = loan_data.loan_status.values
X = loan_data.drop('loan_status', 1).values

In [9]:
from sklearn.linear_model import LogisticRegression

In [25]:
cls = LogisticRegression()

In [11]:
cls.fit(X,Y)

/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
model.score(X, Y)

0.7843827869430198

In [17]:
X.shape

(482378, 14)

In [33]:
X_train = X[:400000,:]

In [34]:
X_test = X[400000:,:]

In [35]:
Y_train = Y[:400000,]
Y_test = Y[400000:,]

In [44]:
Y_test.shape

model = cls.fit(X_train,Y_train)

/home/kevinreilly/codebase/school/github/lending-club-project/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
model.score(X_train, Y_train)

0.7753775

In [46]:
model.score(X_test, Y_test)

0.8274903493651218